# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started. Once you have completed the code you can download the notebook for making a submission.

In [ ]:
import numpy as np
import os
from imageio import imread
from skimage.transform import resize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [ ]:
np.random.seed(30)
import random as rn
rn.seed(30)
from tensorflow import keras
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

**data path: /home/datasets/Project_data**

In [ ]:
train_doc = np.random.permutation(open('/home/datasets/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/home/datasets/Project_data/val.csv').readlines())

batch_size = 35

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [ ]:
#sample x y z invocations
# x = 30 # No. of frames images
# y = 100 # Width of the image
# z = 100 # height

def generator(source_path, folder_list, batch_size,x,y,z):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx =[i for i in range(0, x)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    image = resize(image, (y, z))  # Resizing to (y,z,3)
                    
                    batch_data[folder,idx,:,:,0] =  image[:, :, 0] / 255.0 #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] =  image[:, :, 1] / 255.0 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] =  image[:, :, 2] / 255.0 #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if len(folder_list) % batch_size != 0:
            remaining_folders = folder_list[num_batches * batch_size:]
            batch_size_remaining = len(remaining_folders)

            batch_data = np.zeros((batch_size_remaining,x,y,z,3))
            batch_labels = np.zeros((batch_size_remaining, 5))

            for folder in range(batch_size_remaining):
                folder_name = remaining_folders[folder].split(';')[0]
                imgs = os.listdir(source_path + '/' + folder_name)

                for idx, item in enumerate(img_idx):
                    image = imread(source_path + '/' + folder_name + '/' + imgs[item]).astype(np.float32)
                    image = resize(image, (y, z))

                    batch_data[folder, idx, :, :, 0] = image[:, :, 0] / 255.0
                    batch_data[folder, idx, :, :, 1] = image[:, :, 1] / 255.0
                    batch_data[folder, idx, :, :, 2] = image[:, :, 2] / 255.0

                batch_labels[folder, int(remaining_folders[folder].strip().split(';')[2])] = 1
            yield batch_data, batch_labels


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [ ]:
curr_dt_time = datetime.datetime.now()
# train_path = '/kaggle/working/Project_data/train'
# val_path = '/kaggle/working/Project_data/val'
train_path = '/home/datasets/Project_data/train' ##From Jarvis, same below
val_path = '/home/datasets/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 15 ## default initiali take 15 only
print ('# epochs =', num_epochs)

In [ ]:
# define function for plotting
import matplotlib.pyplot as plt
def plot(history):
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15,4))
    axes[0].plot(history.history['loss'])   
    axes[0].plot(history.history['val_loss'])
    axes[0].legend(['loss','val_loss'])

    axes[1].plot(history.history['categorical_accuracy'])   
    axes[1].plot(history.history['val_categorical_accuracy'])
    axes[1].legend(['categorical_accuracy','val_categorical_accuracy'])

## Model
Here we made the model using different functionalities that Keras provides: Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

####  Model1: 100x100 image with CNN3D

In [ ]:
#Import keras packages

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout, Flatten, BatchNormalization, Activation, Conv3D, MaxPooling3D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import Adam
x=30
y=100
z=100
#Model
model_1 = Sequential()       
model_1.add(Conv3D(8,kernel_size=(3,3,3),input_shape=(30, y, z, 3),padding='same'))
model_1.add(BatchNormalization())
model_1.add(Activation('relu'))

model_1.add(Conv3D(16, (3, 3, 3), padding='same'))
model_1.add(Activation('relu'))
model_1.add(BatchNormalization())
model_1.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_1.add(Conv3D(32, (2, 2, 2), padding='same'))
model_1.add(Activation('relu'))
model_1.add(BatchNormalization())
model_1.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_1.add(Conv3D(64, (2, 2, 2), padding='same'))
model_1.add(Activation('relu'))
model_1.add(BatchNormalization())
model_1.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_1.add(Conv3D(128, (2, 2, 2), padding='same'))
model_1.add(Activation('relu'))
model_1.add(BatchNormalization())
model_1.add(MaxPooling3D(pool_size=(2, 2, 2)))      

# Flatten layer 

model_1.add(Flatten())

model_1.add(Dense(128, activation='relu'))
model_1.add(Dropout(0.5))

model_1.add(Dense(64, activation='relu'))
model_1.add(Dropout(0.5))

#Softmax layer

model_1.add(Dense(5, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [ ]:
from tensorflow.keras.optimizers import Adam
optimiser = Adam(learning_rate=0.01)#write your optimizer
model_1.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_1.summary())

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [ ]:
train_generator = generator(train_path, train_doc, batch_size,x,y,z)
val_generator = generator(val_path, val_doc, batch_size,x,y,z)

In [ ]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.keras'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto',save_freq = 'epoch')

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=4, verbose=1)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit` method to decide the number of next() calls it need to make.

In [ ]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [ ]:
history1=model_1.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, initial_epoch=0)

In [ ]:
plot(history1)

### Model2: 160x160 image with CNN3D

In [ ]:
x=30
y=160
z=160
#write your model here
model_2 = Sequential()       
model_2.add(Conv3D(8,kernel_size=(3,3,3),input_shape=(30, y, z, 3),padding='same'))
model_2.add(BatchNormalization())
model_2.add(Activation('relu'))

model_2.add(Conv3D(16, (3, 3, 3), padding='same'))
model_2.add(Activation('relu'))
model_2.add(BatchNormalization())
model_2.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_2.add(Conv3D(32, (2, 2, 2), padding='same'))
model_2.add(Activation('relu'))
model_2.add(BatchNormalization())
model_2.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_2.add(Conv3D(64, (2, 2, 2), padding='same'))
model_2.add(Activation('relu'))
model_2.add(BatchNormalization())
model_2.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_2.add(Conv3D(128, (2, 2, 2), padding='same'))
model_2.add(Activation('relu'))
model_2.add(BatchNormalization())
model_2.add(MaxPooling3D(pool_size=(2, 2, 2)))      

# Flatten layer 

model_2.add(Flatten())

model_2.add(Dense(128, activation='relu'))
model_2.add(Dropout(0.5))

model_2.add(Dense(64, activation='relu'))
model_2.add(Dropout(0.5))

#Softmax layer

model_2.add(Dense(5, activation='softmax'))

In [ ]:
from tensorflow.keras.optimizers import Adam
optimiser = Adam(learning_rate=0.001)#write your optimizer
model_2.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_2.summary())

In [ ]:
train_generator = generator(train_path, train_doc, batch_size,x,y,z)
val_generator = generator(val_path, val_doc, batch_size,x,y,z)

In [ ]:
history2=model_2.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=20, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, initial_epoch=0,workers=1)

In [ ]:
plot(history2)

### Model3: with 160X160 and batch size 60 (using model2 and new batch size)

In [ ]:
batch_size = 60
train_generator = generator(train_path, train_doc, batch_size,x,y,z)
val_generator = generator(val_path, val_doc, batch_size,x,y,z)

In [ ]:
# The commented code in this cell refers to Experiment 3 in write-up. The code failed due to memory issue.

# history3=model_2.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
#                     callbacks=callbacks_list, validation_data=val_generator, 
#                     validation_steps=validation_steps, class_weight=None, initial_epoch=0)

### Model4: different set of Conv3d filter and Dense layer

In [ ]:
x = 30 # number of frames
y = 120 # image width
z = 120 # image height

# Input all the images sequencial by building the layer with dropouts and batch normalisation

model_4 = Sequential()       
model_4.add(Conv3D(8,kernel_size=(2,2,2),input_shape=(x,y,z,3),padding='same'))
model_4.add(BatchNormalization())
model_4.add(Activation('relu'))

model_4.add(Conv3D(16, (2, 2, 2), padding='same'))
model_4.add(Activation('relu'))
model_4.add(BatchNormalization())
model_4.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_4.add(Conv3D(32, (2, 2, 2), padding='same'))
model_4.add(Activation('relu'))
model_4.add(BatchNormalization())
model_4.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_4.add(Conv3D(64, (2, 2, 2), padding='same'))
model_4.add(Activation('relu'))
model_4.add(BatchNormalization())
model_4.add(MaxPooling3D(pool_size=(2, 2, 2)))

# Flatten layer 

model_4.add(Flatten())

model_4.add(Dense(256, activation='relu'))
model_4.add(Dropout(0.2))

model_4.add(Dense(128, activation='relu'))
model_4.add(Dropout(0.2))

#Softmax layer

model_4.add(Dense(5, activation='softmax'))

In [ ]:
optimiser = Adam(learning_rate=0.01)#write your optimizer
model_4.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_4.summary())

In [ ]:
batch_size = 15
train_generator = generator(train_path, train_doc, batch_size,x,y,z)
val_generator = generator(val_path, val_doc, batch_size,x,y,z)

In [ ]:
history4=model_4.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, initial_epoch=0)

In [ ]:
plot(history4)

### Model5: CNN + LSTM

In [ ]:
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Bidirectional, BatchNormalization, Activation, Dropout, GlobalAveragePooling2D, GlobalAveragePooling3D, ConvLSTM2D
from keras.layers.convolutional import Conv2D, Conv3D, MaxPooling2D, MaxPooling3D
from keras import optimizers


x = 30 # number of frames
y = 120 # image width
z = 120 # image height

model_5 = Sequential()
model_5.add(TimeDistributed(Conv2D(8, (3,3), activation='relu'), input_shape=(x,y,z,3)))
model_5.add(BatchNormalization())
model_5.add(TimeDistributed(Conv2D(16, (3,3), activation='relu')))
model_5.add(BatchNormalization())
model_5.add(ConvLSTM2D(8, kernel_size = 3, return_sequences=False))
model_5.add(BatchNormalization())
model_5.add(TimeDistributed(Dense(64, activation='relu')))
model_5.add(BatchNormalization())
model_5.add(GlobalAveragePooling2D())
model_5.add(Dense(64, activation='relu'))
model_5.add(Dense(5, activation='softmax'))

In [ ]:
optimiser = Adam(learning_rate=0.001)#write your optimizer
model_5.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_5.summary())

In [ ]:
batch_size = 15
train_generator = generator(train_path, train_doc, batch_size,x,y,z)
val_generator = generator(val_path, val_doc, batch_size,x,y,z)

In [ ]:
history5=model_5.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, initial_epoch=0,workers=1)

In [ ]:
plot(history5)

### Model6: TimeDistributed Conv2D + GRU

In [ ]:
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Bidirectional, BatchNormalization, Activation, Dropout, GlobalAveragePooling2D, GlobalAveragePooling3D, ConvLSTM2D
from keras.layers.convolutional import Conv2D, Conv3D, MaxPooling2D, MaxPooling3D
model_6 = Sequential()
model_6.add(TimeDistributed(Conv2D(32, (3,3), activation='relu'), input_shape=(x,y,z,3)))
model_6.add(TimeDistributed( MaxPooling2D((2,2))))
model_6.add(BatchNormalization())
model_6.add(Dropout(0.2))

model_6.add(TimeDistributed(Conv2D(64, (3,3), activation='relu')))
model_6.add(TimeDistributed(MaxPooling2D((2,2))))
model_6.add(BatchNormalization())
model_6.add(Dropout(0.2))

model_6.add(TimeDistributed(GlobalAveragePooling2D()))
model_6.add(TimeDistributed(Dense(64, activation='relu')))
model_6.add(BatchNormalization())
model_6.add(Dropout(0.2))

model_6.add(GRU(128))
model_6.add(BatchNormalization())
model_6.add(Dense(5, activation='softmax'))

In [ ]:
optimiser = Adam(learning_rate=0.001)#write your optimizer
model_6.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_6.summary())

In [ ]:
batch_size = 20
train_generator = generator(train_path, train_doc, batch_size,x,y,z)
val_generator = generator(val_path, val_doc, batch_size,x,y,z)

In [ ]:
history6=model_6.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=25, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, initial_epoch=0)

In [ ]:
plot(history6)

### Model7: Different hyperparamter for CNN and dropout

In [ ]:
x = 30 # number of frames
y = 120 # image width
z = 120 # image height

model_7 = Sequential()
model_7.add(Conv3D(16, (3, 3, 3), padding='same',input_shape=(x,y,z,3)))
model_7.add(Activation('relu'))
model_7.add(BatchNormalization())
model_7.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_7.add(Conv3D(32, (3, 3, 3), padding='same'))
model_7.add(Activation('relu'))
model_7.add(BatchNormalization())
model_7.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_7.add(Conv3D(64, (2, 2, 2), padding='same'))
model_7.add(Activation('relu'))
model_7.add(BatchNormalization())
model_7.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_7.add(Conv3D(128, (2, 2, 2), padding='same'))
model_7.add(Activation('relu'))
model_7.add(BatchNormalization())
model_7.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_7.add(Flatten())
model_7.add(Dense(64,activation='relu'))
model_7.add(BatchNormalization())
model_7.add(Dropout(0.25))

model_7.add(Dense(64,activation='relu'))
model_7.add(BatchNormalization())
model_7.add(Dropout(0.25))

model_7.add(Dense(5,activation='softmax'))

optimiser = Adam(learning_rate=0.002)
model_7.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_7.summary())

In [ ]:
batch_size = 20
train_generator = generator(train_path, train_doc, batch_size,x,y,z)
val_generator = generator(val_path, val_doc, batch_size,x,y,z)

In [ ]:
history7=model_7.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=20, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, initial_epoch=0)

In [ ]:
plot(history7)

### Model8: 160x160 image

In [ ]:
x = 30 # number of frames
y = 160 # image width
z = 160 # image height

model_8 = Sequential()
model_8.add(Conv3D(16, (3, 3, 3), padding='same',input_shape=(x,y,z,3)))
model_8.add(Activation('relu'))
model_8.add(BatchNormalization())
model_8.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_8.add(Conv3D(32, (3, 3, 3), padding='same'))
model_8.add(Activation('relu'))
model_8.add(BatchNormalization())
model_8.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_8.add(Conv3D(64, (2, 2, 2), padding='same'))
model_8.add(Activation('relu'))
model_8.add(BatchNormalization())
model_8.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_8.add(Conv3D(128, (2, 2, 2), padding='same'))
model_8.add(Activation('relu'))
model_8.add(BatchNormalization())
model_8.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_8.add(Flatten())
model_8.add(Dense(64,activation='relu'))
model_8.add(BatchNormalization())
model_8.add(Dropout(0.25))

model_8.add(Dense(64,activation='relu'))
model_8.add(BatchNormalization())
model_8.add(Dropout(0.25))

model_8.add(Dense(5,activation='softmax'))

optimiser = Adam(learning_rate=0.002)
model_8.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_8.summary())

In [ ]:
batch_size = 20
train_generator = generator(train_path, train_doc, batch_size,x,y,z)
val_generator = generator(val_path, val_doc, batch_size,x,y,z)

In [ ]:
history8=model_8.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=20, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, initial_epoch=0)

In [ ]:
plot(history8)

### Model9: Transfer learning

In [ ]:
from keras.applications import mobilenet
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Bidirectional, BatchNormalization, Activation, Dropout, GlobalAveragePooling2D, GlobalAveragePooling3D, ConvLSTM2D
# from keras.layers.convolutional import Conv2D, Conv3D, MaxPooling2D, MaxPooling3D
from tensorflow.keras.layers import Conv3D, ConvLSTM2D,MaxPooling2D, MaxPooling3D
from tensorflow.keras.optimizers import Adam
x = 30 # number of frames
y = 128 # image width
z = 128 # image height
mobilenet_transfer = mobilenet.MobileNet(weights='imagenet', include_top=False,input_shape=(y, z, 3))
model_9 = Sequential()
model_9.add(TimeDistributed(mobilenet_transfer,input_shape=(x,y,z,3)))


model_9.add(TimeDistributed(BatchNormalization()))
model_9.add(TimeDistributed(MaxPooling2D((2, 2))))
model_9.add(TimeDistributed(Flatten()))

model_9.add(GRU(32))
model_9.add(Dropout(0.25))

model_9.add(Dense(32,activation='relu'))
model_9.add(Dropout(0.25))

model_9.add(Dense(5,activation='softmax'))


optimiser = Adam(learning_rate=0.002)
model_9.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
batch_size = 20
train_generator = generator(train_path, train_doc, batch_size,x,y,z)
val_generator = generator(val_path, val_doc, batch_size,x,y,z)

In [ ]:
history9=model_9.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=20, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, initial_epoch=0)

In [ ]:
plot(history9)